# Notes
These are just notes I used for constructing the TFP joints

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
tfd = tfp.distributions
tfb = tfp.bijectors
from tensorflow_probability.python.internal import tensorshape_util

In [2]:
import maxentep

In [3]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]])
var = start / 2
print(start[start > 0])

j = maxentep.recip_norm_mat_dist(start, var)
j.log_prob(j.sample([5]))

[10.  3.  2.  1.]
tf.Tensor([10.  3.  2.  1.], shape=(4,), dtype=float64)
[10 3 2 1]
tf.Tensor([5.  1.5 1.  0.5], shape=(4,), dtype=float64)
[5 1.5 1 0.5]
tfp.distributions.JointDistributionSequential("JointDistributionSequential", batch_shape=[[], []], event_shape=[[4], [4]], dtype=[float64, float64])


<tf.Tensor: shape=(5,), dtype=float64, numpy=array([3.81063614, 5.45469597, 2.39288811, 2.73703578, 2.13950219])>

In [4]:
w = tf.constant(start.astype(np.float32))
w, tf.ones(tf.rank(w))

(<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
 array([[ 0., 10.,  0.,  0.],
        [ 0.,  0.,  3.,  0.],
        [ 0.,  0.,  0.,  2.],
        [ 0.,  0.,  0.,  1.]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>)

In [15]:
i =  tf.keras.Input((1,))
combined = np.stack((start, var))
x  = maxentep.TrainableInputLayer(combined, constraint=maxentep.PositiveMaskedConstraint(combined > 0))(i)

In [18]:
m = tf.keras.Model(inputs=i, outputs=x)
t = m(tf.constant([1., 1.]))
t

<tf.Tensor: shape=(2, 2, 4, 4), dtype=float32, numpy=
array([[[[ 0. , 10. ,  0. ,  0. ],
         [ 0. ,  0. ,  3. ,  0. ],
         [ 0. ,  0. ,  0. ,  2. ],
         [ 0. ,  0. ,  0. ,  1. ]],

        [[ 0. ,  5. ,  0. ,  0. ],
         [ 0. ,  0. ,  1.5,  0. ],
         [ 0. ,  0. ,  0. ,  1. ],
         [ 0. ,  0. ,  0. ,  0.5]]],


       [[[ 0. , 10. ,  0. ,  0. ],
         [ 0. ,  0. ,  3. ,  0. ],
         [ 0. ,  0. ,  0. ,  2. ],
         [ 0. ,  0. ,  0. ,  1. ]],

        [[ 0. ,  5. ,  0. ,  0. ],
         [ 0. ,  0. ,  1.5,  0. ],
         [ 0. ,  0. ,  0. ,  1. ],
         [ 0. ,  0. ,  0. ,  0.5]]]], dtype=float32)>

In [19]:
t[0,0], t[0,1]

(<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
 array([[ 0., 10.,  0.,  0.],
        [ 0.,  0.,  3.,  0.],
        [ 0.,  0.,  0.,  2.],
        [ 0.,  0.,  0.,  1.]], dtype=float32)>,
 <tf.Tensor: shape=(4, 4), dtype=float32, numpy=
 array([[0. , 5. , 0. , 0. ],
        [0. , 0. , 1.5, 0. ],
        [0. , 0. , 0. , 1. ],
        [0. , 0. , 0. , 0.5]], dtype=float32)>)

In [25]:
maxentep.recip_norm_mat_dist(t[0,0], t[0,1]).sample(5)

tf.Tensor([10.  3.  2.  1.], shape=(4,), dtype=float32)
[10 3 2 1]
tf.Tensor([5.  1.5 1.  0.5], shape=(4,), dtype=float32)
[5 1.5 1 0.5]
tfp.distributions.JointDistributionSequential("JointDistributionSequential", batch_shape=[[], []], event_shape=[[4], [4]], dtype=[float32, float32])


<tf.Tensor: shape=(5, 4, 4), dtype=float32, numpy=
array([[[0.80142534, 0.19857466, 0.        , 0.        ],
        [0.        , 0.5488671 , 0.4511329 , 0.        ],
        [0.        , 0.        , 0.59409314, 0.40590686],
        [0.        , 0.        , 0.        , 1.        ]],

       [[0.89689827, 0.10310172, 0.        , 0.        ],
        [0.        , 0.65878713, 0.34121287, 0.        ],
        [0.        , 0.        , 0.04668474, 0.95331526],
        [0.        , 0.        , 0.        , 1.        ]],

       [[0.8927919 , 0.10720808, 0.        , 0.        ],
        [0.        , 0.82513076, 0.17486925, 0.        ],
        [0.        , 0.        , 0.73671097, 0.26328903],
        [0.        , 0.        , 0.        , 1.        ]],

       [[0.9412596 , 0.05874038, 0.        , 0.        ],
        [0.        , 0.7965391 , 0.20346089, 0.        ],
        [0.        , 0.        , 0.25761694, 0.74238306],
        [0.        , 0.        , 0.        , 1.        ]],

       [[0.94

In [22]:
i =  tf.keras.Input((1,))
l = maxentep.recip_norm_mat_layer(i, start, var, 'test')

Tensor("test-dist_1/GatherNd:0", shape=(None,), dtype=float32)
Tensor("test-dist_1/GatherNd_1:0", shape=(None,), dtype=float32)
tfp.distributions.JointDistributionSequential("test_dist_1_JointDistributionSequential", batch_shape=[[], ?], event_shape=[[?], [?]], dtype=[float32, float32])


In [23]:
m = tf.keras.Model(inputs=i, outputs=l)

In [24]:
m(tf.constant([1.]))

tf.Tensor([10.  3.  2.  1.], shape=(4,), dtype=float32)
[10 3 2 1]
tf.Tensor([5.  1.5 1.  0.5], shape=(4,), dtype=float32)
[5 1.5 1 0.5]
tfp.distributions.JointDistributionSequential("model_6_test_dist_JointDistributionSequential", batch_shape=[[], []], event_shape=[[4], [4]], dtype=[float32, float32])


InvalidArgumentError: ConcatOp : Ranks of all input tensors should match: shape[0] = [1,4] vs. shape[1] = [4] [Op:ConcatV2] name: concat